In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## 탐지 수행 함수

In [3]:
def is_cellphone_detected(image_path, model, confidence_threshold=0.3, iou_threshold=0.5):
    """YOLO 모델을 사용하여 핸드폰 탐지 수행 (핸드폰만 탐지)"""
    image = cv2.imread(image_path)
    if image is None:
        return None  # 이미지 로드 실패 시 None 반환

    # YOLO 모델 실행
    results = model(image, conf=confidence_threshold, iou=iou_threshold)
    predictions = results[0].boxes.data  # 감지된 객체 정보

    # 예측된 객체 중 핸드폰이 감지되었는지 확인
    for prediction in predictions:
        class_id = int(prediction[5])  # 5번은 클래스 ID 정보가 있다
        if class_id == 0:  # 핸드폰 클래스 ID가 0일 경우
            return 1  # 핸드폰이 감지되면 1 반환

    return 0  # 핸드폰이 감지되지 않으면 0 반환


## 배치를 받아 탐지 수행 함수(탐지 수행 함수를 받는다)

In [4]:
def is_cellphone_detected_batch(image_paths, model):
    """배치로 이미지를 처리하여 핸드폰 탐지 여부를 리턴"""
    results = []
    for image_path in image_paths:
        detected = is_cellphone_detected(image_path, model)
        results.append(detected)
    return results

## 데이터 일부만 사용하는 함수

In [5]:
def process_folder(root_folder, output_folder, output_filename, model_path, confidence_threshold=0.3, iou_threshold=0.5, batch_size=36):
    data = []

    for subdir, _, files in os.walk(root_folder):
        folder_name = os.path.basename(subdir)

        print(f"모델 로드 중: {folder_name}")
        model = YOLO(model_path)  # YOLO 모델 로드
        image_paths_batch = []

        for file in files:
            if file.endswith(".jpg"):
                image_path = os.path.join(subdir, file)
                image_paths_batch.append(image_path)

                if len(image_paths_batch) >= batch_size:
                    detected_results = is_cellphone_detected_batch(image_paths_batch, model)

                    for image_path, detected in zip(image_paths_batch, detected_results):
                        relative_path = os.path.relpath(image_path, root_folder)
                        folder_parts = relative_path.split(os.sep)
                        big_folder = folder_parts[0]
                        image_name = folder_parts[-1]
                        data.append([big_folder, image_name, detected])
                    print(f"배치 처리 완료: {len(image_paths_batch)}개의 이미지 탐지 완료.")
                    image_paths_batch = []

        if image_paths_batch:
            detected_results = is_cellphone_detected_batch(image_paths_batch, model)
            for image_path, detected in zip(image_paths_batch, detected_results):
                relative_path = os.path.relpath(image_path, root_folder)
                folder_parts = relative_path.split(os.sep)
                big_folder = folder_parts[0]
                image_name = folder_parts[-1]
                data.append([big_folder, image_name, detected])

    if data:
        df = pd.DataFrame(data, columns=["video_name", "frame", "Phone_Detected"])
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, output_filename)
        df.to_excel(output_path, index=False)
        print(f"탐지 결과가 {output_path}에 저장되었습니다.")
    else:
        print("해당 조건을 만족하는 폴더가 없어 탐지를 수행하지 않았습니다.")


## 실행

In [ ]:

# /content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized/
# /content/drive/MyDrive/DMS/data/YawDD dataset/YawDD_adjusted_frames_resized

data_path = "/content/drive/MyDrive/DMS/data/DMD/dmd_frames_resized"
input_folder = os.path.join(data_path, 'Normal')

save_path = '/content/drive/MyDrive/DMS/phone_detection_results/resized_0427'
output_folder = save_path
output_filename = os.path.join(output_folder, 'DMD_normal_detection.xlsx')

model_path = '/content/drive/MyDrive/phone_finetunning_0415/exp12/weights/best.pt'

process_folder(input_folder, output_folder, output_filename, model_path)

Streaming output truncated to the last 5000 lines.

0: 224x224 (no detections), 109.5ms
Speed: 2.0ms preprocess, 109.5ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 119.6ms
Speed: 3.6ms preprocess, 119.6ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 95.8ms
Speed: 1.0ms preprocess, 95.8ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 93.8ms
Speed: 0.9ms preprocess, 93.8ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 123.3ms
Speed: 1.0ms preprocess, 123.3ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 135.6ms
Speed: 1.0ms preprocess, 135.6ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 103.7ms
Speed: 1.4ms preprocess, 103.7ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

0: 224x